In [1]:
from pathlib import Path
from exiftool import ExifToolHelper
from media_tool.move import IMAGE_EXTENSIONS
from multiprocessing import Pool
from tqdm.auto import tqdm

/home/menma/Repos/media-tool/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/mnt/orico/photo/fix-date"

In [3]:
def get_images(path):
    """Organizes image and video files into subfolders based on their creation date."""
    source_path = Path(path)

    if not source_path.exists() or not source_path.is_dir():
        print("Source folder does not exist or is not a directory.")
        return

    # Update to recursively find all media files using rglob
    media_files = []
    for ext in IMAGE_EXTENSIONS:
        media_files.extend(source_path.rglob(f"*{ext}"))
        media_files.extend(source_path.rglob(f"*{ext.upper()}"))

    return media_files

In [4]:
files = get_images(path)
example_file = files[0]
example_file

PosixPath('/mnt/orico/photo/fix-date/2022-03-30/DSCF2094.JPG')

In [5]:
with ExifToolHelper() as et:
    metadata_list = et.get_metadata(example_file)
    metadata = metadata_list[0]  # Get the first (and only) metadata dictionary
metadata

{'SourceFile': '/mnt/orico/photo/fix-date/2022-03-30/DSCF2094.JPG',
 'ExifTool:ExifToolVersion': 13.0,
 'File:FileName': 'DSCF2094.JPG',
 'File:Directory': '/mnt/orico/photo/fix-date/2022-03-30',
 'File:FileSize': 18805430,
 'File:FileModifyDate': '2025:03:19 16:48:56+00:00',
 'File:FileAccessDate': '2025:03:19 16:48:56+00:00',
 'File:FileInodeChangeDate': '2025:03:19 16:48:56+00:00',
 'File:FilePermissions': 100777,
 'File:FileType': 'JPEG',
 'File:FileTypeExtension': 'JPG',
 'File:MIMEType': 'image/jpeg',
 'File:ExifByteOrder': 'II',
 'File:ImageWidth': 7728,
 'File:ImageHeight': 5152,
 'File:EncodingProcess': 0,
 'File:BitsPerSample': 8,
 'File:ColorComponents': 3,
 'File:YCbCrSubSampling': '2 1',
 'EXIF:Make': 'FUJIFILM',
 'EXIF:Model': 'X-T5',
 'EXIF:Orientation': 8,
 'EXIF:XResolution': 72,
 'EXIF:YResolution': 72,
 'EXIF:ResolutionUnit': 2,
 'EXIF:Software': 'Digital Camera X-T5 Ver2.10',
 'EXIF:ModifyDate': '2024:03:30 05:44:05',
 'EXIF:Artist': '',
 'EXIF:YCbCrPositioning': 2,

In [6]:
def create_date_updates(file_path: str, target_year: int) -> dict:
    """
    Creates a dictionary of date field updates for the given file, changing the year
    while preserving all other date components.

    Args:
        file_path: Path to the image file
        target_year: The year to set in the date fields

    Returns:
        Dictionary containing the field updates
    """
    # Date fields we want to modify
    date_fields = [
        "EXIF:ModifyDate",
        "EXIF:DateTimeOriginal",
        "EXIF:CreateDate",
        "XMP:CreateDate",
        "XMP:ModifyDate",
        "XMP:DateCreated",
        "IPTC:DigitalCreationDate",
        "IPTC:DateCreated",
        "Composite:SubSecCreateDate",
        "Composite:SubSecDateTimeOriginal",
        "Composite:SubSecModifyDate",
        "Composite:DateTimeCreated",
        "Composite:DigitalCreationDateTime",
    ]

    updates = {}

    # Read current dates from file
    with ExifToolHelper() as et:
        metadata = et.get_metadata(file_path)
        if metadata:
            metadata = metadata[0]  # Get first (and only) item

            for field in date_fields:
                if field in metadata:
                    current_date_str = str(metadata[field])
                    try:
                        updates[field] = str(target_year) + current_date_str[4:]
                    except ValueError:
                        print(
                            f"Could not parse date for field {field}: {current_date_str}"
                        )
                        continue

    return updates


def apply_date_updates(file_path: str, updates: dict) -> bool:
    """
    Applies the date updates to the specified file using ExifTool.

    Args:
        file_path: Path to the image file
        updates: Dictionary of field updates from create_date_updates()

    Returns:
        Boolean indicating success
    """
    try:
        with ExifToolHelper() as et:
            # Convert updates to ExifTool command format
            params = ["-overwrite_original"]  # Overwrite original file
            for field, value in updates.items():
                params.extend([f"-{field}={value}"])

            # Apply the updates
            et.execute(*params, file_path)
            return True
    except Exception as e:
        print(f"Error updating file {file_path}: {str(e)}")
        return False

In [7]:
def process_file(file):
    """
    Process a single file with date updates.
    Helper function for multiprocessing.
    """
    updates = create_date_updates(file, 2024)
    success = apply_date_updates(file, updates)
    return file, success


def process_all_files(files, num_processes=4):
    """
    Process all files using multiprocessing pool
    """
    total_files = len(files)

    with Pool(processes=num_processes) as pool:
        # Create progress bar
        with tqdm(total=total_files, desc="Updating dates") as pbar:
            # Process files in parallel
            for file, success in pool.imap_unordered(process_file, files):
                status = "✓" if success else "✗"
                pbar.set_postfix_str(f"Last: {Path(file).name} [{status}]")
                pbar.update()


# Run the processing
process_all_files(files, num_processes=8)

Updating dates: 100%|██████████| 4465/4465 [10:04<00:00,  7.38it/s, Last: DSCF0008.RAF [✓]]                                          
